# RAG application built on gemini 

In [1]:
%pip install langchain langchain_community langchain-google-genai python-dotenv streamlit langchain_experimental sentence-transformers langchain_chroma langchainhub pypdf rapidocr-onnxruntime

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [19]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("FEWD Unit-3.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
data

# from langchain_community.document_loaders import PyPDFLoader

# pdf_files = ["yolov9_paper.pdf", "FEWD Unit-3.pdf"]

# # Initialize an empty list to store the loaded data
# data = []

# # Loop through the list of PDFs and load each one
# for pdf_file in pdf_files:
#     loader = PyPDFLoader(pdf_file)
#     temp_data = loader.load()  # Load the PDF
#     data.append(temp_data)  # Add the loaded document to the list

[Document(metadata={'source': 'FEWD Unit-3.pdf', 'page': 0}, page_content="React Event Handling  \nEvent handling is a fundamental aspect of building interactive user interfaces in React. Events such as \nclicks, key presses, mouse movements, etc., trigger actions within React components. Properly handling \nthese events is crucial for  creating responsive and interactive applications.  \nReact components can listen and respond to various events similar to HTML event handling.  \nEvents are handled using event handlers, which are functions that are executed when a specific event \noccurs.  \nHere a re some examples in React demonstrating how to listen and respond to various events . \n1. Handling Click Events:  \nimport React from 'react';  \n \nclass ClickExample extends React.Component {  \n  handleClick() {  \n    alert('Button clicked!');  \n  } \n \n  render() {  \n    return ( \n      <button onClick={this.handleClick}>Click me</button>  \n    ); \n  } \n} \n \nexport default Cli

In [20]:
len(data)

21

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  38


In [22]:
docs[7]

Document(metadata={'source': 'FEWD Unit-3.pdf', 'page': 3}, page_content='/> \n    ); \n  } \n} \n \nexport default FocusExample;  \nIn this example, the handleFocus method is called when the input field receives focus, a nd the handleBlur \nmethod is called when the input field loses focus.')

In [23]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [24]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [25]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is new in yolov9?")


In [26]:
len(retrieved_docs)

10

In [28]:
print(retrieved_docs[5].page_content)

target task, it also overcomes the problems encountered by
mask modeling. The proposed PGI mechanism can be ap-
plied to deep neural networks of various sizes and is more
general than the deep supervision mechanism, which is only
suitable for very deep neural networks.
In this paper, we also designed generalized ELAN
(GELAN) based on ELAN [65], the design of GELAN si-
multaneously takes into account the number of parameters,
computational complexity, accuracy and inference speed.
This design allows users to arbitrarily choose appropriate
computational blocks for different inference devices. We
combined the proposed PGI and GELAN, and then de-
signed a new generation of YOLO series object detection
system, which we call YOLOv9. We used the MS COCO
dataset to conduct experiments, and the experimental results
verified that our proposed YOLOv9 achieved the top perfor-
mance in all comparisons.
We summarize the contributions of this paper as follows:


In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response = rag_chain.invoke({"input": "what is new in YOLOv9?"})
print(response["answer"])

YOLOv9 introduces two new components: **Programmable Gradient Information (PGI)** for enhanced gradient flow and **Generalized ELAN (GELAN)**, a computationally efficient network architecture. These enhancements allow YOLOv9 to achieve state-of-the-art performance on object detection tasks. 



In [33]:
response = rag_chain.invoke({"input": "what is Auxiliary Supervision?"})
print(response["answer"])

Auxiliary Supervision is a common technique in deep learning, often used to enhance the training process. It involves adding extra prediction layers (deep supervision) or using relevant metadata to guide the intermediate layers of a model. This helps to speed up convergence and improve the model's robustness. 



In [34]:
response = rag_chain.invoke({"input": "Explain about OnClick event in React?"})
print(response["answer"])

In React, the `onClick` event is used to handle click events on HTML elements like buttons, divs, etc. When a user clicks the element, the function assigned to the `onClick` attribute is executed. This function can then perform any desired action, such as updating the UI, fetching data, or triggering other events. 



In [35]:
response = rag_chain.invoke({"input": "Give Sample code on OnClick event Handler in React?"})
print(response["answer"])

```
import React from 'react';

class ClickExample extends React.Component {
  handleClick() {
    alert('Button clicked!');
  }

  render() {
    return (
      <button onClick={this.handleClick}>Click me</button>
    );
  }
}

export default ClickExample;
```

This code defines a button that, when clicked, will display an alert message. The `handleClick` function is bound to the button's `onClick` event, ensuring it's triggered upon clicking. 



In [37]:
response = rag_chain.invoke({"input": "Give Sample code on OnKeyPress in React?"})
print(response["answer"])

```javascript
import React from 'react';

class KeyboardExample extends React.Component {
  handleKeyPress = (event) => {
    console.log('Key pressed:', event.key);
  }

  render() {
    return (
      <input
        type="text"
        onKeyPress={this.handleKeyPress}
        placeholder="Type something..."
      />
    );
  }
}

export default KeyboardExample;
```

This code defines a component with an input field. The `onKeyPress` event handler is attached to the input, and when a key is pressed, it logs the pressed key to the console. 

